In [ ]:
!~/downloads/google-cloud-sdk/bin/gcloud iam service-accounts create access2

In [ ]:
!~/downloads/google-cloud-sdk/bin/gcloud projects add-iam-policy-binding anu-tier-1 --member "serviceAccount:access2@anu-tier-1.iam.gserviceaccount.com" --role "roles/DialogflowAPIClient"

In [ ]:
!~/downloads/google-cloud-sdk/bin/gcloud iam service-accounts keys create key2.json --iam-account access1@anu-tier-1.iam.gserviceaccount.com


In [1]:


# !~/downloads/google-cloud-sdk/bin/gcloud beta auth application-default  login
!export GOOGLE_APPLICATION_CREDENTIALS="/Users/Tom/.config/gcloud/key2.json"
#token = !gcloud auth application-default print-access-token

from bs4 import BeautifulSoup
import requests
import re
import lxml.html as lh
from urllib.request import urlretrieve
import pandas as pd
import urllib
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import textract

In [ ]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [ ]:
"""
f = open('All_Programs.csv','r')
final_course_list=[]
content = f.readlines()
for c in content:
    if(c[0].isupper() and c[1].isupper() and c[2].isupper()):
        final_course_list.append(c[0:8])
"""
program_df = pd.read_csv('All_Programs.csv')
program_df.columns =list(map(lambda x: x.strip(), program_df.columns))
#f.close()

In [ ]:
def is_honours(program):
    return "H" in program

YEARS = range(2018,2020)
for year in YEARS:
    for program in program_df['Code']:
        #if(not is_honours(program)):
        #    continue
        if(len(program) > 5):
            continue
        print(program)
        main_page = requests.get("https://programsandcourses.anu.edu.au/"+str(year)+"/program/"+program)
        soup = BeautifulSoup(main_page.content, 'html.parser')
        course_dict=dict()
        course_dict["year"] = year
        required = dict()
        required["x_from_here"] = [] 
        required["x_from_category"] = []
        required["one_from_m/m/s"] = []
        for title in soup.find_all("span",class_ = "intro__degree-title__component"):
            course_dict["name"] = title.get_text()
        for units in soup.find_all("li",class_ = "degree-summary__requirements-units"):
            course_dict["units"] = [int(s) for s in units.get_text().split() if s.isdigit()][0]
        
        if(is_honours(program)):
            curr_node = ""
            curr_num = None
            for s in soup.find_all("p"):
                
                st = str(s)
                if("units from completion" in st) and not ('elective' in st):
                        curr_node = st[:-3].split(">")[1]
                        curr_num =st.split(" ")[0]
                        print(curr_node)
                        
                if("margin-left: 80px;" in st):
                    print(st)
                    for statement in str(s).split(">"):
                        if(curr_node == ""):
                            break
                        if(curr_node == "mms")and ("href" in statement):
                            required["one_from_m/m/s"].append(statement[-9:])
                            continue
                        if(curr_node != "") and ("href" in statement):
                            required["x_from_here"][-1]["courses"].append(statement[-9:])
                            if(curr_num == '6'):
                                break
                        if("following" in statement) and (("major" in statement) or ("minor" in statement) or ("specialisation" in statement)):
                            curr_node = "mms"
                            continue
                        if("units from completion" in statement) and not ('elective' in statement):
                            curr_node = statement[:-3]
                            curr_num =statement.split(" ")[0]
                            if("List 1" in statement):
                                required["x_from_category"].append({"num":curr_num,"code":['List 1']})

                            required["x_from_here"].append({"num":curr_num,"courses":[]})
        else:
            for s in soup.find_all("div",{"id": "study"}):
                curr_node = ""
                curr_num = None
                for statement in str(s).split(">"):
                    if(curr_node == "mms")and ("href" in statement):
                        required["one_from_m/m/s"].append(statement[-9:])
                        continue
                    if(curr_node != "") and ("href" in statement):
                        required["x_from_here"][-1]["courses"].append(statement[-9:])
                        if(curr_num == '6'):
                            break
                    if("following" in statement) and (("major" in statement) or ("minor" in statement) or ("specialisation" in statement)):
                        curr_node = "mms"
                        continue
                    if("units from completion" in statement) and not ('elective' in statement):
                        curr_node = statement[:-3]
                        curr_num =statement.split(" ")[0]
                        if("List 1" in statement):
                            required["x_from_category"].append({"num":curr_num,"code":['List 1']})

                        required["x_from_here"].append({"num":curr_num,"courses":[]})



            course_dict["required"] = required
            print(course_dict)
            #s.split(">")
            #bs = BeautifulSoup(s.content,'html.parser')
            #for p in bs.find_all("p"):
            #    print(p)
        
# look for a regex which is like > (some number) units from completion of a course from the following list.
# Then get all the HRefs in the proceeding list.
# format in xml
# readd for all degrees


In [ ]:
course_codes = list(pd.read_csv("courses.csv")['\n                    Code\n                '])
course_names = list(pd.read_csv("courses.csv")['\n                    Title\n                '])
course_descriptions=[]
course_codes=final_course_list
for i in range(0,len(final_course_list)):
    if(course_codes[i] in seen):
        break
    seen.add(i)
    course=course_codes[i]
    print("Downloading reports for:",course)
    main_page = requests.get("https://programsandcourses.anu.edu.au/2018/course/"+course)
    soup = BeautifulSoup(main_page.content, 'html.parser')
    desc=""
    req=""
    outcomes=""
    req_description=""
    full_text=soup.get_text()
    for s in soup.find_all("div","introduction"):
        desc = s.get_text()
    for s in soup.find_all("div","requisite"):
        req=parse_prereq(s.get_text())
        
        req_description=s.get_text()
        
    for s in soup.find_all("ol"):
        outcomes = s.get_text()    

    course_descriptions.append([course,course_names[i],desc,outcomes,req,req_description,full_text])


In [ ]:
import re

for program in program_df['Code']:
    main_page = requests.get("https://programsandcourses.anu.edu.au/program/"+program)
    soup = BeautifulSoup(main_page.content, 'html.parser')
    doc = ""
    for para in soup.find_all('p'):
        doc += " "+para.text
    doc = ' '.join(s for s in doc.split() if not s.isdigit())
    regex = re.compile('[^a-zA-Z]')
    doc =regex.sub(' ', doc)
    doc = ''.join(e for e in doc if e.isalnum() or e==' ' or e=="." or e=="," or e=="\n")
    url = 'https://dialogflow.googleapis.com/v2beta1/projects/anu-tier-1/agent/knowledgeBases/OTM2NzIwNTc0OTk1MzkyMTAyNA/documents'
    payload = """{
              "name":"",
              "displayName": \""""+program+"""\",
              "mimeType": "text/plain",
              "knowledgeTypes": [
                "EXTRACTIVE_QA"
              ],

              "content": \""""+doc+"""\",
            }""" 
    headers = {
               'Authorization': 'Bearer '+token[0]
    }

    print(requests.post(url, data=payload, headers=headers).text)


In [31]:
!~/downloads/google-cloud-sdk/bin/gcloud auth print-access-token


ya29.c.El8GBl0IVoUhYDKLqAtl_f5yMMmmWnJEB6kWcaJYZRoMC9O-Ux97okyictxTlKkV1c8TxJYugiDKxSUI0XIzTqI2KlirBIjTQyyi72K5wK7KXP0kfX3lTMoSoZAWFTJSLw


In [68]:
token = ["ya29.c.El8GBl0IVoUhYDKLqAtl_f5yMMmmWnJEB6kWcaJYZRoMC9O-Ux97okyictxTlKkV1c8TxJYugiDKxSUI0XIzTqI2KlirBIjTQyyi72K5wK7KXP0kfX3lTMoSoZAWFTJSLw"]
token = !~/downloads/google-cloud-sdk/bin/gcloud auth print-access-token
base_id = "OTM2NzIwNTc0OTk1MzkyMTAyNA"
from unidecode  import unidecode

def answer_question(q):
    q=q.replace(" ","+")
    main_page = requests.get("https://www.google.com/search?q="+q+"+site:programsandcourses.anu.edu.au")
    soup = BeautifulSoup(main_page.content, 'html.parser')
    main_page = requests.get('http'+soup.find('h3', class_ = 'r').find('a').get('href').split('http')[1].split('&sa')[0])
    soup = BeautifulSoup(main_page.content, 'html.parser')
    doc=""
    for para in soup.find_all('p'):
        doc += "\n"+unidecode(para.text)
    #doc = ' '.join(s for s in doc.split() if not s.isdigit())
    #regex = re.compile('[^a-zA-Z.,?]')
    #doc =regex.sub(' ', doc)
    #doc = ''.join(e for e in doc if e.isalnum() or e==' ' or e=="." or e=="," or e=="\n" or e=="?")
    doc = doc.replace("\"","")
    doc_id = add_to_knowledge_base(doc,base_id)
    
    print(match_intent(q))
    delete_from_knowledge_base(base_id,doc_id)
    #delete_from_knowledge_base(base_id,doc_id)
#dialogflow-xfakkc@anu-tier-1.iam.gserviceaccount.com
#b6573caeb88881d7988e1bc0e8b3a4719d55f481

answer_question("what do I learn in bachelor of engineering")

{
  "name": "projects/anu-tier-1/operations/ks-add_document-MzMzMDU4Nzg0NjMwMDkyNTk1Mg"
}

{
  "responseId": "87e5def4-331e-4877-bf69-8ec4a7cfd327",
  "queryResult": {
    "queryText": "what+do+I+learn+in+bachelor+of+engineering",
    "parameters": {},
    "allRequiredParamsPresent": true,
    "intent": {
      "name": "projects/anu-tier-1/agent/intents/e357fc86-ca4a-4e2b-b5fe-3cdb07db2a3b",
      "displayName": "Knowledge.KnowledgeBase.OTM2NzIwNTc0OTk1MzkyMTAyNA"
    },
    "intentDetectionConfidence": 0.27764508,
    "languageCode": "en",
    "knowledgeAnswers": {
      "answers": [
        {
          "source": "projects/anu-tier-1/knowledgeBases/OTM2NzIwNTc0OTk1MzkyMTAyNA/documents/OTE2MDYwMzExNzA0ODI5OTUy",
          "answer": "If you are an international student you must always be full time. IF YOU ARE COMMENCING IN JULY YOU SHOULD SEND AN EMAIL TO studentadmin.cecs anu.edu.au FOR ADVICE ABOUT YOUR ENROLMENT OR YOU SHOULD ATTEND AN ENROLMENT ADVICE SESSION AT THE UNIVERSITY IN TH

In [61]:
#try to find on dialogflow

#automatically authenticate, return the code
#implicit()

#create a new knowledge base, including finding out the key, if possible
def create_knowledge_base(name):
    url = 'https://dialogflow.googleapis.com/v2beta1/projects/anu-tier-1/agent/knowledgeBases'
    
    payload = """{
                  "name": "",
                  "displayName": \""""+name+"""\"
                }""" 
    headers = {
               'Authorization': "Bearer "+token[0]
    }
    res = requests.post(url, data=payload,headers=headers).text
    return json.loads(res)["name"].split("/")[3]

#add the document to the knowledge base
def add_to_knowledge_base(doc,base):
    url = 'https://dialogflow.googleapis.com/v2beta1/projects/anu-tier-1/knowledgeBases/'+base+'/documents'
    payload = """{
              "name":"",
              "displayName": "testing doc 2",
              "mimeType": "text/html",
              "knowledgeTypes": [
                "EXTRACTIVE_QA"
              ],

              "content": \""""+doc+"""\",
            }""" 
    headers = {
               'Authorization': "Bearer "+token[0]
    }
    
    res = requests.post(url, data=payload,headers=headers).text
    print(res)
    return json.loads(res)["name"].split("-")[4]
    

    
def delete_from_knowledge_base(base,doc_id):
    url = 'https://dialogflow.googleapis.com/v2beta1/projects/anu-tier-1/agent/knowledgeBases/'+base+'/documents/'+doc_id
    payload = """{
              "name":\""""+doc_id+"""\",
              }"""
    headers = {
               'Authorization': "Bearer "+token[0]
    }
    print(requests.delete(url, data=payload,headers=headers).text)


In [28]:
def match_intent(s):
    s = "\""+s+"\""
    url = 'https://dialogflow.googleapis.com/v2beta1/projects/anu-tier-1/agent/sessions/ddfab5c3-64bd-a138-1e5e-894614e7d7bb:detectIntent'
    payload = """{
        "queryInput":{
            "text":{
                "text":%s,
                "languageCode":"en"
                }
            },
        "queryParams":{
            "timeZone":"Australia/Sydney"
        }
    }""" % s
    headers = {
               'Authorization': "Bearer "+token[0]
        }

    return requests.post(url, data=payload, headers=headers).text

In [ ]:
#d =!~/downloads/google-cloud-sdk/bin/gcloud auth application-default print-access-token


In [ ]:
"bearer "+token[0]

In [ ]:
!~/downloads/google-cloud-sdk/bin/gcloud auth print-access-token